# This script compares the shape of a group of samples
## Input a dictionary with the location of the samples and their name.
## Also, add the name of the histogram and the binning.

In [9]:
from histogramHelpers import normalization
import ROOT as r
import numpy as np

rootDirectory = "/Users/diegomac/Documents/HEP/VBF-Analysis/Zll/Z-peak/SR/"
## QCDjj shapes ###
if (False):

    channel = "Zll_"
    samples = {"MadGraph@NLO":rootDirectory+channel+"MGNLORW.root",
               "MadGraph@LO": rootDirectory+channel+"MGRW.root",
               "Sherpa2.2.11": rootDirectory+channel+"SherpaNLORW.root",
               "Sherpa2.2.1": rootDirectory+channel+"SherpaRW.root"}

    colors = {"MadGraph@NLO":r.kBlue,
               "MadGraph@LO": r.kGreen,
               "Sherpa2.2.11": r.kRed,
               "Sherpa2.2.1": r.kBlack}

    styles = {"MadGraph@NLO":1,
               "MadGraph@LO": 1,
               "Sherpa2.2.11": 1,
           "Sherpa2.2.1": 1}
    
## EWjj shapes ###
if (True):

    channel = "Signal_"
    samples = {"PowHeg+Pythia8":rootDirectory+channel+"PoPy.root",
               "Sherpa2.2.11": rootDirectory+channel+"Sherpa.root",}

    colors = {"PowHeg+Pythia8":r.kBlue,
               "Sherpa2.2.11": r.kBlack}

    styles = {"PowHeg+Pythia8":1,
           "Sherpa2.2.11": 1}
    
    

histogramName = "mass_jj"
bining = np.array([   0.,  250.,  500.,  750., 1000., 1250., 1500., 2000., 2500., 3000., 4000., 5000.])
normalisationConstant = 250.0

defaultSample = "Sherpa2.2.11"

In [18]:
r.TH1.AddDirectory(False)
r.gStyle.SetOptStat(0)

canvas = r.TCanvas("canvas")
canvas.cd()

pad1 = r . TPad (" pad1 "," pad1 " ,0 ,0.45 ,1 ,1)
pad1.SetTopMargin(0.03)
pad1.SetRightMargin(0.03)
pad1.SetLeftMargin(0.08)
pad1.SetBottomMargin(0.0)
pad1.Draw ()
pad1.cd ()
pad1.SetLogy()
pad1.cd()



for sampleName,sampleFile in samples.items():

    file = r.TFile.Open(sampleFile,"READ")
    histogram = file.Get(histogramName)
    histogram.SetDirectory(0)
    histogram = histogram.Rebin(len(bining)-1,sampleName,bining)
    
    integral = histogram.Integral(1,-1)
    histogram.Scale(1.0/integral)
    
    histogram.SetLineColor(colors[sampleName])
    histogram.SetLineStyle(styles[sampleName])
    histogram.SetTitle(sampleName)
    histogram.GetYaxis().SetRangeUser(0.00005,0.6)
    
    normalization([histogram],normalisationConstant)
    histogram.Draw("HIST same")
    
pad1.BuildLegend()

pad1.FindObject("PowHeg+Pythia8").SetTitle("EWjj Shapes")
pad1.FindObject("PowHeg+Pythia8").SetTitleOffset(0.2)
pad1.Update()

canvas.cd()
pad2 = r . TPad (" pad2 "," pad2 " ,0 ,0.10 ,1 ,0.45)
pad2.SetRightMargin(0.03)
pad2.SetLeftMargin(0.08)
pad2.SetTopMargin(0)
pad2.SetBottomMargin(0.18)
pad2.Draw ()
pad2.cd()

#Construct default sample
file = r.TFile.Open(samples[defaultSample],"READ")
defaultHistogram = file.Get(histogramName)
defaultHistogram.SetDirectory(0)
defaultHistogram = defaultHistogram.Rebin(len(bining)-1,samples[defaultSample],bining)
integral = defaultHistogram.Integral(1,-1)
defaultHistogram.Scale(1.0/integral)
normalization([defaultHistogram],normalisationConstant)
file.Close()

dummy = defaultHistogram.Clone()
dummy.Divide(dummy)
dummy.SetTitle("")
dummy.Draw("hist same")

dummy.GetYaxis().SetRangeUser(0.8,1.5)


for sampleName,sampleFile in samples.items():
    
    if sampleName!=defaultSample:

        file = r.TFile.Open(sampleFile,"READ")
        histogram = file.Get(histogramName)
        histogram.SetDirectory(0)
        histogram = histogram.Rebin(len(bining)-1,sampleName,bining)

        integral = histogram.Integral(1,-1)
        print(integral)
        histogram.Scale(1.0/integral)

        histogram.SetLineColor(colors[sampleName])
        histogram.SetFillColorAlpha(colors[sampleName],0.7)
        histogram.SetLineStyle(styles[sampleName])
        histogram.SetTitle(sampleName)

        normalization([histogram],normalisationConstant)
        
        histogram.Divide(defaultHistogram)
        histogram.Draw("E5 same")
        
dummy.GetYaxis().SetTitle("PowHeg+Pythia8/"+defaultSample)
dummy. GetYaxis (). SetTitleSize (0.07)
dummy. GetYaxis (). SetTitleOffset (.25)

dummy.GetXaxis().SetTitle("Mjj(GeV)")
dummy. GetXaxis (). SetTitleSize (0.10)
dummy. GetXaxis (). SetTitleOffset (0.85)
dummy. GetXaxis (). SetLabelSize(0.07)


canvas.Print("EWjj.pdf")
    

3667.842409133911


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file EWjj.pdf has been created
